https://github.com/KevinLiao159/MyDataSciencePortfolio/blob/af84b7162e320ee72622265b3b2857a8dc772f55/movie_recommender/src/knn_recommender.py#L224

In [48]:
import pandas as pd
import numpy as np
from scipy import sparse
from tqdm.notebook import tqdm
from sklearn.neighbors import NearestNeighbors

from mapk import mapk
from train_test_split import holdout

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [49]:
# articles = pd.read_csv('../data/articles.csv')
# customers = pd.read_csv('../data/customers.csv')
transactions_train = pd.read_csv('../data/transactions_train.csv')

In [50]:
# from sklearn.model_selection import train_test_split
transactions_train['t_dat'] = pd.to_datetime(transactions_train['t_dat'])
# transactions_train = transactions_train.query('t_dat > "2019-08-16"')

# train

In [51]:
train, test = holdout(transactions_train)

In [52]:
# удалим из обучающей выборки пользователей, которые есть в тестовой
test_users = test[['customer_id']]

train_users = set(
    (
        pd.merge(train[['customer_id']], test_users, indicator=True, how='outer')
        .query('_merge=="left_only"')
        .drop('_merge', axis=1)
    ).customer_id
)

train = train.query('customer_id in @train_users')

In [53]:
def create_mappings(df, column):
    '''
    create_mappings(df, column)
        Создаёт маппинг между оригинальными ключами словаря и новыми порядковыми.
        
        Parameters
        ----------
        df : pandas.DataFrame
            DataFrame с данными.
            
        column : str
            Название колонки, содержащей нужны ключи. 
        
        Returns
        -------
        code_to_idx : dict
            Словарь с маппингом: "оригинальный ключ" -> "новый ключ".
        
        idx_to_code : dict
            Словарь с маппингом: "новый ключ" -> "оригинальный ключ".
    '''
    
    code_to_idx = {}
    idx_to_code = {}
    
    # for idx, code in enumerate(df[column].to_list()):
    for idx, code in enumerate(set(df[column])):
        code_to_idx[code] = idx
        idx_to_code[idx] = code
        
    return code_to_idx, idx_to_code


def map_ids(row, mapping):
    '''
    Вспомогательная функция
    '''
    return mapping[row]

In [54]:
%%time

# словари с маппингами, нужно для tocsr матрицы
user_to_idx, idx_to_user = create_mappings(train, 'customer_id')
item_to_idx, idx_to_item = create_mappings(train, 'article_id')

# собираем матрицу взаимодействий 
U = train['customer_id'].apply(map_ids, args=[user_to_idx]).values
I = train['article_id'].apply(map_ids, args=[item_to_idx]).values

# тут пока бинарная маска будет лежать для упрощения
# можно попробовать поиграться с ценами
values = np.ones(train.shape[0])
    
interactions = sparse.coo_matrix(
    (values, (U, I)), 
    shape=(len(train['customer_id'].unique()), len(train['article_id'].unique())), 
    dtype=np.float64
).tocsr()


CPU times: user 40.6 s, sys: 1.49 s, total: 42.1 s
Wall time: 42.1 s


In [55]:
model = NearestNeighbors(n_neighbors=20, metric='cosine', n_jobs=-1)
model.fit(interactions)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=20)

In [60]:
def make_predict_most_pop(train=train):
    most_pop = train.query('t_dat > "2020-09-05"')\
        .groupby('article_id').size()\
        .sort_values(ascending=False)[:12].index.values
    return list(most_pop) 

def make_predict(x, model, interactions=interactions):
    user = x.customer_id
    
    if user in user_to_idx:
        idx = user_to_idx[user]

        distances, indices = model.kneighbors(interactions[idx], n_neighbors=20)

        neighbor_users = [idx_to_user[i] for i in indices[0][1:]]
    
        # query('t_dat > "2020-08-16"') - покупки за последний месяц
        local_pop = train.loc[train.customer_id.isin(neighbor_users)]\
                        .query('t_dat > "2020-08-16"')\
                        .groupby('article_id').size()\
                        .sort_values(ascending=False)[:12].index.values
        return list(local_pop)
    else:
        most_pop = make_predict_most_pop()
        return most_pop

In [61]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [62]:
%%time
t = test.iloc[:10_000].copy()
t['predict'] = t.progress_apply(make_predict, 
                                     model=model,
                                     interactions=interactions,  
                                     axis=1)

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 37min 33s, sys: 21min 7s, total: 58min 41s
Wall time: 55min 3s


In [63]:
mapk(t['target'].to_list(), t['predict'].to_list())

0.007418226151824365